In [8]:
import tensorflow as tf
import os
import requests
import io
import PIL
import PIL.Image
import math
import pathlib
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, Input, Dense, Reshape, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import pickle
import pandas as pd



In [9]:
batch_size = 8 # 每⼀批所处理的图⽚数量
img_height = 256 # 图⽚⾼度，单位为像素
img_width = 256 # 图⽚宽度，单位为像素

channels = 3

train_dir = './datasets/tikuimage'

TF_SERVING_BASE_URL = 'http://localhost:8501/'

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   validation_split=0.2)
 
training_set = train_datagen.flow_from_directory(
     train_dir,
     target_size = (img_height, img_width),
     batch_size = batch_size,
     class_mode = 'input',
     subset = 'training',
     shuffle=True)
 
validation_set = train_datagen.flow_from_directory(
     train_dir,
     target_size = (img_height, img_width),
     batch_size = batch_size,
     class_mode = 'input',
     subset = 'validation',
     shuffle=False)



Found 2502 images belonging to 11 classes.
Found 623 images belonging to 11 classes.


In [15]:
#latent_space_model = keras.experimental.load_from_saved_model('models/image_autoencoder_2.h5').get_layer('latent_space').output
latent_space_model = tf.keras.models.load_model('models/image_autoencoder_2.h5').get_layer('latent_space').output


#latent_space_model = Model(
#                      autoencoder.input, 
#                      autoencoder.get_layer('latent_space').output)
 

In [21]:
X = []
indices = []

#Tqdm 是一个快速，可扩展的Python进度条，可以在 Python 长循环中添加一个进度提示信息，用户只需要封装任意的迭代器 tqdm(iterator)。
 
for i in tqdm(range(len(os.listdir('./datasets/tikuimage/new')))):
  try:
    img_name = os.listdir('./datasets/tikuimage')[i]
    img = load_img('./datasets/tikuimage/new/{}'.format(img_name), 
                   target_size = (256, 256))
    img = img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    pred = latent_space_model.predict(img)
    pred = np.resize(pred, (16))
    X.append(pred)
    indices.append(img_name)
    print(i)
 
  except Exception as e:
    print('in exception:')
    print(img_name)
    print(e)

    # Export the embeddings
embeddings = {'indices': indices, 'features': np.array(X)}
print(embeddings)
#pickle.dump(embeddings, open('./image_embeddings.pickle', 'wb')


<ipython-input-21-2c1c60c706f1>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(os.listdir('./datasets/tikuimage/new')))):


in exception:
temp
[Errno 21] Is a directory: './datasets/tikuimage/new/temp'
in exception:
.DS_Store
cannot identify image file <_io.BytesIO object at 0x7f89b5c4fbd0>
in exception:
mt212
[Errno 21] Is a directory: './datasets/tikuimage/new/mt212'
in exception:
mt215
[Errno 21] Is a directory: './datasets/tikuimage/new/mt215'
in exception:
mt214
[Errno 21] Is a directory: './datasets/tikuimage/new/mt214'
in exception:
mt213
[Errno 21] Is a directory: './datasets/tikuimage/new/mt213'
in exception:
new
[Errno 2] No such file or directory: './datasets/tikuimage/new/new'
in exception:
mt216
[Errno 21] Is a directory: './datasets/tikuimage/new/mt216'
in exception:
.ipynb_checkpoints
[Errno 21] Is a directory: './datasets/tikuimage/new/.ipynb_checkpoints'
in exception:
mt211
[Errno 21] Is a directory: './datasets/tikuimage/new/mt211'
in exception:
mt218
[Errno 21] Is a directory: './datasets/tikuimage/new/mt218'
in exception:
mt210
[Errno 21] Is a directory: './datasets/tikuimage/new/mt210'


In [5]:
print(train_ds.class_names)
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
(8, 256, 256, 3)
(8,)


In [6]:
num_classes = 5
model = tf.keras.Sequential([ # 根据需要调整模型结构
layers.experimental.preprocessing.Resizing(img_height, img_width),
tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
   
])

In [7]:
model.compile(
optimizer='adam',
loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=['accuracy'])

In [8]:
model.fit(
train_ds,
validation_data=val_ds,
epochs=1
)
print(model.evaluate(val_ds))

92/92 [==============================] - 6s 62ms/step - loss: 1.0600 - accuracy: 0.6172
[1.0599966049194336, 0.6171662211418152]


In [9]:
tf.keras.models.save_model(
model,
'./models/image/1/', # ./models为tensorflow-serving的模型根⽬录
overwrite=True,
include_optimizer=True,
save_format=None,
signatures=None,
options=None
)

INFO:tensorflow:Assets written to: ./models/image/1/assets


In [10]:
def test_image_model(test_dir, code, batch_size=10): 
  imgs = []
  codes = []
  imgdir = os.path.join(test_dir, str(code)) 
  for i in pathlib.Path(imgdir).glob('./*.jpg'): 
    img = PIL.Image.open(i)
    pixels = np.array(img)
    imgs.append(pixels.tolist()) 
  for i in range(int(math.ceil(len(imgs)/batch_size))):
    req_data = json.dumps({
'     inputs': imgs[i*batch_size:(i+1)*batch_size],
    }) 
    #http://localhost:8501/v1/models/fashion_mnist:predict
    response = requests.post(TF_SERVING_BASE_URL+'v1/models/image/versions/1:predict', # 根据部署地址填写
    data=req_data,
    headers={"content-type": "application/json"})
    if response.status_code != 200:
      raise RuntimeError('Request tf-serving failed: ' + response.text)
    resp_data = json.loads(response.text) 
    if 'outputs' not in resp_data \
         or type(resp_data['outputs']) is not list:
      raise ValueError('Malformed tf-serving response')
    codes.extend(np.argmax(resp_data['outputs'], axis=1).tolist())
    return codes

In [11]:
codes = test_image_model(val_ds, 0)
print('类别0的准确率', 1 - round(np.sum(codes)/len(codes),4))
codes = test_image_model(val_ds, 1)
print('类别1的准确率', round(np.sum(codes)/len(codes),4))

TypeError: expected str, bytes or os.PathLike object, not BatchDataset